# Best Prof Dokumentation
## Fragestellung:
Können durch die Auswertung der Daten aus dem UnivIS die Platzierungen des "Best Prof Awards Informatik 2022" richtig vorhergesagt werden?
## Zur Verfügung stehende Daten:
Wir haben für den Zeitraum 2010 - 2019 die jeweils besten drei Platzierungen des "Best Prof Awards Informatik" von der Seite der Fachschaften Informatik & Mathematik der Uni Kiel.
Außerdem stehen uns die Daten aus dem UnivIS (Informationssystem der Universität Kiel) seit dem Sommersemester 2000 in Form einer SQLite Datenbank zur Verfügung.
## Vorgehen:
In der Datei bestProf.py haben wir die benötigten Daten mit Hilfe von Funktionen gesammelt und aufbereitet.
Daher importieren wir diese Datei als Erstes.

In [52]:
%reload_ext autoreload
%autoreload 2

import bestProf
import sqlalchemy
db = sqlalchemy.create_engine('sqlite:///')

Anschließend erstellen wir ein Data Frame, das alle Module enthält, die jeweils den Gewinnerplatzierungen pro Jahr zugeordnet werden können.
Dabei ist zu beachten, dass der Award im Sommer verliehen wird. Zu einem Awardjahr gehören also Module des aktuellen Sommersemesters sowie des im vorherigen Jahr beginnenden Wintersemesters.

In [53]:
df_lecture = bestProf.create_lecture_df(db)

Damit wir die Daten mit allen Modulen vergleichen können, erstellen wir noch ein Data Frame, dass für jedes Jahr von 2010 bis 2019 alle angebotenen Modulnamen mit den Namen der Lehrpersonen, sowie Teilnehmerzahl, ECTS und Semester speichert. Dies benötigen wir allerdings erst etwas später.

In [54]:
df_all_moduls = bestProf.create_df_all_moduls(db)

### Anzahl der Module:
Aus 'df_lecture' errechnen wir die Anzahl der Module, die pro Awardjahr stattgefunden haben.

In [22]:
df_count_lecture = bestProf.calculates_num_of_moduls(df_lecture, df_all_moduls)

Die Anzahl der gehaltenen Module lässt sich pro Jahr und Platzierung in einem Balkendiagramm darstellen.

In [23]:
bestProf.visualization_num_of_moduls(df_count_lecture).show()

Hier ist zu erkennen, dass die Gesamtanzahl der gehaltenen Module im Laufe der Jahre leicht sinkt.
Der gelbe Balken zeigt die durchschnittliche Anzahl von Modulen einer Lehrperson im Institut für Informatik. Diese Anzahl hebt sich nicht sonderlich ab.
Es ist für die einzelnen Platzierungen also kein richtiger Trend erkennbar.
Dafür visualisieren wir die Anzahl pro Platzierung und Jahr noch mit einem Scatter Plot mit einer Trendlinie, die die Berechnung "lowess" verwendet.

In [24]:
bestProf.visualization_num_of_moduls_trend(df_count_lecture).show()

Durch die drei Trendlinien lässt sich eine differenzierte Aussage für die einzelnen Platzierungen treffen:
1. Platz: Die Anzahl der Module sinkt kontinuierlich über die Jahre und ist 2019 bei einem Wert von 5,28.
2. Platz: Hier ist kein richtiger Trend erkennbar. Die Linie stagniert in den Jahren 2010 - 2018 im Bereich 5 - 7. Nur 2019 ist der Wert bei ca. 4,4
3. Platz: Bis 2012 gab es einen leichten Anstieg, ab 2014 sinkt die Anzahl wieder und bleibt ab 2017 im Bereich 7,6 - 7,8 .
4. Durchschnitt aller Module: Hier ist eine deutliche Abgrenzung zu den Best Prof Platzierungen ab dem Jahr 2015 sichtbar. Der Wert sinkt zwar auch leicht, ist aber insgesamt deutlich über der Modulanzahl der Best Profs

### Häufige Module:
Welche Module aus der Awardmodulliste sind denn die häufigsten? Ist ein Sieg durch ein bestimmtes Modul garantiert?
Wir gruppieren und zählen dafür die einzelnen Module aus df_lecture.

In [25]:
bestProf.visualization_lecture(df_lecture).show()

Auffallend ist das 'Hardwarepraktikum'. Dieses kommt insgesamt achtmal vor und ist damit der Spitzenreiter. Allerdings wurde dieses Modul auch jedes Semester von SS10 - SS18 angeboten. Es war also 4 Jahre unter den Top 3.
Gleiches gilt für das 'Masterabschlussseminare Echtzeitsysteme / Eingebettete Systeme' und für das 'Masterseminar - Programmiersprachen und Programmiersysteme'. Diese wurde ebenfalls häufig im Sommer- und Wintersemester angeboten.
Zu der Arbeitsgemeinschaft und der Weiterbildung finden sich in der Moduldatenbank keine weiteren Informationen.
Insgesamt werden von den genannten Modulen laut ModulDB nur noch 'Computersysteme', 'Logik in der Informatik' und das 'Masterseminar - Programmiersprachen und Programmiersysteme' angeboten.
Daher hilft uns diese Auswertung nicht für die Vorhersage des Best Prof Awards 2022 weiter.

### Anzahl ECTS:
Wenn die erfolgreichsten Module nicht mehr angeboten werden, die Modulanzahl also insgesamt im Jahresverlauf sinkt, steigt dann der Arbeitsaufwand und somit die Anzahl der Punkte?
Dafür erstellen wir uns einen Data Frame, der die ECTS pro Jahr, Semester und Platzierung beinhaltet.

In [26]:
df_ects = bestProf.create_ects_df(df_lecture, db)

Mit einem Scatter Plot sehen wir uns nun erstmal die Verteilung der ECTS im Jahresverlauf nach Winter- und Sommersemester getrennt an.

In [ ]:
bestProf.visualization_ects_scatterplot_summer(df_ects).show()

In [ ]:
bestProf.visualization_ects_scatterplot_winter(df_ects).show()

Hierbei ist lediglich zu erkennen, dass sich die Verteilung der ECTS pro Sommer- und Wintersemester bei Platz 1 ähnelt.
Die anderen beiden Platzierungen haben im jeweiligen Jahr unterschiedliche Verteilungen pro Semester.
Wir sehen uns nun also eine Gesamtübersicht pro Jahr als Lineplot an, bei dem wir vorher den Mittelwert der ECTS pro Jahr berechnet haben und diese zusätzlich mit dem Mittelwert aller Module vergleichen können.
Für den Wert aller Module wurden zuerst jeweils die einzelnen ECTS aller Module einer Person pro Jahr summiert und durch die Anzahl der Module geteilt. Anschließend haben wir dann den Durchschnitt aller Personen pro Jahr berechnet.

In [27]:
bestProf.visualization_ects_lineplot_year(df_ects, df_all_moduls).show()

Die ECTS-Gesamtanzahl der Awardmodule hat je nach Platzierung Ausreißer in unterschiedlichen Jahren, sinkt aber im Jahresverlauf.
Auffällig ist, dass Platz 1 alle 3 - 4 Jahre um ca. 1 Punkt ansteigt, um in den anschließenden beiden Jahren wieder um erst ca. 1,5 und dann ca. 0,5 Punkte zu sinken. Um daraus aber ein konkretes Muster abzuleiten, bräuchten wir mehr Daten mit einem größeren Zeitverlauf.
Der 2. Platz hingegen hat kein Muster, aber sein niedrigster Mittelwert 4 im Jahr 2017 steigert sich innerhalb von 2 Jahren auf seinen höchsten Mittelwert von 8.
Platz 3 steigt bis 2012 und sinkt seitdem mit einem signifikanten Ausreißer im Jahr 2018 kontinuierlich.
Unsere erste Vermutung, dass weniger Module mehr Arbeitsaufwand und mehr ECTS bedeuten, können wir also nicht bestätigen.
Erkennbar ist aber, dass der Durchschnitt aller Module im Vergleich zu den Best Prof Modulen weniger stark schwankt und sich durchgehend im Bereich 5,8 bis 6,9 befindet.

### Anzahl englischsprachiger Module:
Anschließend haben wir uns gefragt, ob bei den Modulen eine bestimmte Lehrsprache bevorzugt wird.
Dafür erstellen wir erstmal ein Data Frame, dass die Sprache pro Modul pro Semester und Platzierung speichert.

In [ ]:
df_language = bestProf.create_lecture_language(df_lecture, db)

Anschließend zählen wir, wie viele Module deutsch- und wie viele englischsprachig sind und visualisieren das Ergebnis mit einem Balkendiagramm.

In [ ]:
df_count_language = bestProf.count_lecture_language(df_language)

In [ ]:
bestProf.visualization_language_bar_plot(df_count_language).show()

Diese Datenauswertung bringt uns bei unserer Fragestellung nicht viel weiter. Von 254 angesehenen Modulen sind lediglich 6 englischsprachig, davon eins 2017, vier 2018 und eins 2019.
Da stellt sich die Frage, wie viele Module überhaupt mit englischer Lehrsprache angeboten werden:
Dafür erstellen wir uns ein Data Frame mit sämtlichen Daten aus dem "Modulinformationssystem Informatik".
Dies wurde mit der Datei "moduldbparser.py" erledigt und von uns bereits als csv Datei gespeichert.

In [ ]:
df_modul_db = bestProf.create_df_modul_db()

Wir gruppieren alle Module, die in dem Modulinformationssystem angegeben wurden nach Sprache und zählen diese.

In [ ]:
bestProf.visualization_language_modul_db(df_modul_db).show()

Um die Prozentzahlen besser einordnen zu können, berechnen wir auch noch die Gesamtverteilung der Best Prof Modulsprachen.

In [ ]:
bestProf.visualization_language_pie(df_count_language).show()

Es sind insgesamt fast 20 % aller Module in der Informatik englischsprachig. Aber lediglich 2,5 % der Best Prof Module sind englischsprachig.
Wir können damit lediglich die Aussage treffen, dass englischsprachige Module nicht gänzlich ausgeschlossen sind, aber insgesamt auch nicht sehr häufig vorkommen.
Auch hier benötigt man mehr Jahre, um einen Trend feststellen zu können.

### Erwartete Teilnehmerzahl:
Wir hatten bereits festgestellt, dass die Anzahl der Module sinkt. Steigt dann die erwartete Teilnehmerzahl an? Braucht eine Platzierung eine Mindestanzahl an Teilnehmern, um gewählt zu werden?
Auch hier erstellen wir ein Data Frame, welches die erwartete Teilnehmerzahl aller Module pro Jahr und Platzierung summiert.

In [28]:
df_participants = bestProf.create_num_of_participants(df_lecture, db)

Die Summe visualisieren wir dann nach Plätzen getrennt mit einem Lineplot.

In [47]:
bestProf.visualization_participant_lineplot(df_participants, df_all_moduls).show()

Die Auswertung zeigt, dass die erwartete Teilnehmerzahl partiell schwankt, aber jeder Platz lediglich zwei Jahre mit mehr als 400 erwarteten Teilnehmern hat.
Das Minimum der erwarteten Teilnehmerzahl liegt bei 30 im Jahr 2017. So richtig aussagekräftig ist das noch nicht.
Die durchschnittliche summierte erwartete Teilnehmerzahl aller Module pro Lehrperson im Institut für Informatik ist im Vergleich zu den Best Prof Modulen eher konstant um die 300.
Daher berechnen wir als Nächstes die Durchschnittsteilnehmerzahl pro Jahr und pro Modul.

In [29]:
bestProf.visualization_participant_average(df_participants, df_count_lecture).show()

Jede Platzierung hat pro Jahr also eine ungefähre erwartete Teilnehmerzahl von 330,5 und pro Modul von 39,68

### Verteilung der Institutionen:
Nun wollen wir die zugehörigen Arbeitsgruppen zu den Modulen ansehen. Gibt es da Häufungen oder eine breite Verteilung durch alle Arbeitsgruppen des Instituts?
Dazu erstellen wir als Erstes wieder ein Data Frame, dass pro Jahr und Platzierung den Namen der Arbeitsgruppe mit der Anzahl beinhaltet.

In [48]:
df_orgname = bestProf.create_orgname(df_lecture, db)

Um aus den Daten eine vernünftige Visualisierung zu bekommen, müssen wir diese noch aufbereiten.
Aus optischen Gründen wurde die Arbeitsgruppe "Algorithmische Optimale Steuerung - CO₂-Aufnahme des Meeres" von uns gekürzt zu "Algorithmische Optimale Steuerung".
Ebenso haben wir das "Institut für Experimentelle und Angewandte Physik (Sektion Physik)" umbenannt in "Institut für Experimentelle & Angewandte Physik".

Bis zum 01.10.2018 gab es die Arbeitsgruppe "Technische Informatik". Seit dem 01.10.2018 wurde daraus die Arbeitsgruppe "Verteilte Systeme".
Somit haben wir zur besseren Übersicht & Auswertung alle Module der "Technischen Informatik" der AG "Verteilte Systeme" zugeordnet und stellen diese AG also schon dar, bevor es sie gab.


In [49]:
bestProf.visualization_orgname_all(df_orgname).show()

Die Gesamtübersicht zeigt, dass "Echtzeitsysteme / Eingebettete Systeme", "Programmiersprachen und Übersetzerkonstruktion", "Theoretische Informatik" und "Verteilte Systeme" sich klar absetzten mit einer Häufigkeit im Bereich 34 - 63. Alle anderen Arbeitsgruppen bleiben unter 15 zugehörigen Modulen.
Es fällt auf, dass die eben 4 genannten Module Phasen hatten, bei denen sie mehrere Jahre (3- 4) hintereinander in die Top 3 gekommen sind.
Für eine genauere Auswertung erstellen wir ein Balkendiagramm, dass jedes Jahr nach Platzierungen aufgeteilt darstellt.

In [ ]:
bestProf.visualization_orgname_place(df_orgname).show()

Hier zeigt sich, dass Module, die mehrere Jahre hintereinander dabei sind, keine "Einstiegsplatzierung" haben.
Die Arbeitsgruppe "Verteilte Systeme" ist z.B. seit 2016 in den Top 3. Eingestiegen ist die AG mit Platz 1 und war insgesamt dreimal auf dem 1. Platz und ein Mal auf dem dritten.
Ebenso lässt sich erkennen, dass die zugeordnete Modulanzahl keine Garantie für den 1. Platz ist. Die Anzahl variiert zwischen den einzelnen Platzierungen im Jahresverlauf.
Zusätzlich fällt auf, dass 2012 das einzige Jahr ist, in dem eine Arbeitsgruppe bei mehreren Platzierungen vorkommt. Aber auch da hat jeder Platz zusätzlich noch eine "eigene" Arbeitsgruppe.
Es schaffen also mindestens 3 und maximal 4 unterschiedliche Arbeitsgruppen in die Top 3.

### Beliebte Vorlesungszeiten:
Können wir einen Trend bei den Vorlesungszeiten finden? Sind Profs von montags 8 Uhr Vorlesungen benachteiligt?
Wir erstellen uns ein Data Frame, dass die einzelnen Vorlesungszeiten enthält.

In [56]:
df_modul_time = bestProf.create_modul_time(df_lecture, db)

Anschließend visualisieren wir erst die Tage mit zwei unterschiedlichen Diagrammen.

In [57]:
bestProf.visualization_modul_time_days_pie(df_modul_time).show()

In [58]:
bestProf.visualization_modul_time_days(df_modul_time).show()

Der häufigste Vorlesungstag, der vorkommt, ist mit 26,4 % Mittwoch, dicht gefolgt von Dienstag und Donnerstag.
Montag, Freitag und Samstag schneiden nicht so gut ab.
In der nach Platzierung sortierten Visualisierung ist kein Favorit für eine einzelne Platzierung zu erkennen.
Nun sehen wir uns die Startzeiten der Vorlesungen an.

In [59]:
bestProf.visualization_modul_time_hours_pie(df_modul_time).show()

In [ ]:
bestProf.visualization_modul_time_hours(df_modul_time).show()

Die häufigste Startzeit ist 10:15 Uhr mit 17,9 %. Erstaunlich für alle "Langschläfer" ist, dass 08.15 Uhr mit 11,7 % an zweiter Stelle ist.
Wie bei den Tagen lässt sich auch für die Platzierungen keine favorisierte Startzeit erkennen.

### Klausurterminverteilung im Prüfungszeitraum:
Haben die Prüfungstermine eine sichtbare Auswirkung auf die Wahl des Best Profs? Kommen bestimmte Prüfungszeiten unter den Top 3 gehäuft vor?
Wir werten die Prüfungstermine aus und vergleichen diese mit den Prüfungszeiträumen der jeweiligen Semester. Dann erstellen wir eine Liste mit zwei Data Frames.
Im ersten speichern wir pro Jahr und Platzierung die einzelnen Prüfungen mit den jeweils verbleibenden Tagen zur Prüfung seit Prüfungszeitraumbeginn.
Der zweite sammelt alle Prüfungen, die außerhalb der Prüfungszeiträume stattfinden.

In [ ]:
df_exam = bestProf.create_exam_date(df_lecture, db)

Zuerst sehen wir uns die Prüfungen außerhalb der Prüfungszeiträume an.

In [ ]:
bestProf.visualization_exam_date_no_period(df_exam).show()

Insgesamt sind es 55 Prüfungen außerhalb des PZ. Aber jedes Jahr ist mindestens ein Modul dabei.
Im Schnitt liegen 5,5 Module pro Jahr außerhalb des Prüfungszeitraums.
Interessanter sind aber die Prüfungen im Prüfungszeitraum.

In [ ]:
bestProf.visualization_exam_date_in_period(df_exam).show()

106 Prüfungen in den Prüfungszeiträumen stehen uns zur Auswertung zur Verfügung.
Wir haben jeweils für Platzierung pro Jahr den Mittelwert gebildet und mit einem Scatterplot mit 'Lowess' Trendlinie dargestellt.
Platz 1 und 2 haben ihre Prüfungstermine recht konstant zwischen 4 und 7 Tagen nach Beginn des Prüfungszeitraumes.
Platz 3 schwankt anfänglich, aber steigt seit 2013 kontinuierlich an und war zuletzt bei einem Abstand von 11 Tagen zum Beginn des PZ.

### Anzahl Pflichtmodule:
Wir sehen uns nun noch an, ob die Awardmodule Pflichtmodule sind.
Dafür prüfen wir, ob das Modul in der ModulDB als Pflichtmodul geführt wird und speichern dies in einem Data Frame und visualisieren dies anschließend.

In [ ]:
df_required_modul = bestProf.create_required_moduls(df_modul_db, df_lecture)

In [ ]:
bestProf.visualization_required_modul(df_required_modul).show()

Die Auswertung zeigt, dass 2019 das erste Jahr ist, bei dem mehr Pflichtmodule als sonstige Module dabei sind.
In allen anderen Jahren ist die Anzahl sonstiger Module mehr als die doppelte Anzahl der Pflichtmodule.

## Fazit:
### Wer wurde nach unserer Auswertung Best Prof 2022?
Insgesamt haben wir viele unterschiedliche Aspekte analysiert und können daraus einige Erkenntnisse gewinnen.
Zur besseren Übersicht hier eine Zusammenfassung:
- Vorlesungstage: am beliebtesten Mittwoch, alternativ auch Dienstag und Donnerstag
- Vorlesungsstartzeiten: am beliebtesten 10 Uhr, alternativ 8 und 16 Uhr
- beteiligte Institutionen/Arbeitsgruppen: Anzahl 3 - 4, "Verteilte Systeme" ist sehr wahrscheinlich, ansonsten auch häufig oder zuletzt dabei: "Numerik und Optimierung", "Didaktik der Informatik", "Algorithmen und Komplexität", "Programmiersprachen & Übersetzerkonstruktion", "Theoretische Informatik".
- erwartete Teilnehmerzahl: pro Person in einem Jahr: 330.5, pro Modul: 39.68
- durchschnittliche ECTS: Platz 1: 5 oder 7, Platz 2: >8, Platz 3: < 4.5
- Modulanzahl pro Person:
          1. Platz: Die Anzahl der Module sinkt kontinuierlich über die Jahre und ist 2019 bei einem Wert von 5,28.
          2. Platz: Hier ist kein richtiger Trend erkennbar. Die Linie stagniert in den Jahren 2010 - 2018 im Bereich 5 - 7. Nur 2019 ist der Wert bei ca. 4,4
          3. Platz: Bis 2012 gab es einen leichten Anstieg, ab 2014 sinkt die Anzahl wieder und bleibt ab 2017 im Bereich 7.6 - 7.8 .

Wir entscheiden uns für die Vorlesungstage, Vorlesungsstartzeiten, Institutionen und ECTS, da diese Werte ziemlich konkret und gut umsetzbar sind, um damit unsere Vorhersage des Best Prof Awards 2022 zu versuchen. Die Modulanzahl und erwartete Teilnehmerzahl lassen wir erstmal weg, da diese Faktoren durch die Übergangsphase zwischen der neuen und alten Fachprüfungsordnung nicht gut einzuordnen sind.
Dafür haben wir eine Funktion, die mithilfe einer SQl-Anfrage, für das Wintersemester 2021/2022 und das Sommersemester 2022 anhand unserer Vorgaben alle Lehrpersonen mit Vor- und Nachnamen ausgibt.

In [50]:
prof = bestProf.prediction_best_prof_2022(db)
print(prof)

{('Landsiedel', 'Olaf'), ('Langfeld', 'Barbara'), ('Mühling', 'Andreas')}


Unsere Funktion gibt uns drei Lehrpersonen zurück.
Wir haben damit Prof. Dr. Olaf Landsiedel, Dr. Barbara Langfeld und Prof. Dr. Andreas Mühling als Best Profs 2022 bestimmt.
Als Nächstes sehen wir uns die Werte der einzelnen Profs an.

Olaf Landsiedel:
`Inf-ITSec5: IT-Sicherheit`:  Teilnehmer: 80, ECTS: 3
`infCN-01a: Computer Networks`: Teilnehmer: 450, ECTS: 7
`infDS-01a: Distributed Systems`: Teilnehmer: 40, ECTS: 8
`Inf-ITSec5: IT-Sicherheit`: Teilnehmer: 90, ECTS: 3
`infMPVVS-01a: Master project - Networked and Distributed Systems`: Teilnehmer: 8, ECTS: 10
`infSemDS-01a: Seminar Networked and Distributed Systems`: Teilnehmer: 5, ECTS: 5
Organisation: Verteilte Systeme
→ 6 Module mit insgesamt 863 erwarteten Teilnehmern, also pro Modul 134.6 erwartete Teilnehmer

Barbara Langfeld:
`infMathA2F-01: Mathematik für die Informatik A (2F) ONLINE`: keine erwarteten Teilnehmer, ECTS: 6
`ONLINE: Inf-Math-A: Mathematik für die Informatik A`: Teilnehmer: 450, ECTS: 8
`Inf-Math-B: Mathematik für die Informatik B`: Teilnehmer: 250,  ECTS: 8
Organisation: Numerik und Optimierung
→ 3 Module mit insgesamt 700 erwarteten Teilnehmern, also pro Modul 233.3 erwartete Teilnehmer

Andreas Mühling:
`infEInf-01a: Einführung in die Informatik`: Teilnehmer: 700,  ECTS: 8
`Inf-I1-2FNF: Informatik I (2F/NF)`: keine erwarteten Teilnehmer,  ECTS: 8
`Inf-FD-LeLeG: Lehren und Lernen im Fach Informatik - Grundlagen`: Teilnehmer: 10, ECTS: 3
`infMLSU-01a: Maschinelles Lernen in Schule und Unterricht`: Teilnehmer: 15, ECTS: 6
Organisation: Didaktik der Informatik
→ 5 Module mit insgesamt 725, also pro Modul 181.25 erwartete Teilnehmer

Der ECTS Durchschnitt liegt bei 6, 7.33 und 6.25 für die einzelnen Profs und damit auch im erwarteten Bereich.
Die erwartete Teilnehmerzahl hingegen passt nicht zu unseren Durchschnittswerten und liegt deutlich darüber.
Die Modulanzahl passt ungefähr zu der durchschnittlichen Modulanzahl der letzten Best Profs pro Jahr.

Insgesamt können wir mit den Werten keine genaue Platzierung angeben.

### Wer wurde wirklich Best Prof 2022?

Platz 1 wurde Dr. Barbara Langfeld, Platz 2 Prof. Dr. Olaf Landsiedel und Platz 3 Prof. Dr. Thomas Wilke.
Mit unserer Auswertung haben wir somit zwei von drei Best Profs des Jahres 2022 richtig vorhersagen können.